<h3 align="center" style="font-family: Georgia, Times, 'Times New Roman', serif">I. Requirements</h3>

* Body Definitions
    * $r_0$
    * $g_0$
    * $\mu$
    * $\psi_0$
* Atmosphere Definitions
    * $\rho_0$
    * $H$
    * $C_{T, i}(h)$ polynomial coefficients of temperature profile
    * $\gamma_g$
    * $R_g$
* Vehicle Definitions (single stage)
    * Aerodynamic characteristics:
        * $C_{D, i}(M^2, \cos{(\alpha)})$ polynomial coefficients of drag surface
        * $C_{L, i}(M^2, \cos{(\alpha)})$ polynomial coefficients of lift surface
        * $A_{ref}$
    * Engine Characteristics:
        * $F_{SL}$
        * $F_{vacc}$ 
        * $Isp_{SL}$
        * $Isp_{vacc}$
        * $F_{\%min}$
        * Possible additition:
            * Isp_func(h, p, ...)
            * F_func(h, p, ...)
    * Mass Characteristics
        * $m_0$
        * $m_f$
    * Control Limits
        * $\dot{\gamma}_{max}$ Max yaw rate [rad/s]
        * $\dot{\beta}_{max}$ Max pitch rate [rad/s]
        * $\dot{f}_{max}$ Max thottle rate [s^-1]
    * Aerodynamic Limits
        * $\alpha_{max}$ may be 0 dim or 1 dim 
        * $q_{max}$ may be 0 dim or 1 dim  
* Problem Definitions
    * Initial and final state can be specified by either:
        * Stationary ground position:
            * Lat
            * Lng
            * $v_{\epsilon}$
            * $h$
        * Orbital parameters:
            * $h_a$ or $a$
            * $h_p$ or $e$
            * $i$
            * $\Omega$
            * $\omega$
        * Directly using the kinematic state vector:
            * $[r, \theta, \phi, v_r, \omega, \psi]$
    * Guess trajectory can be calculated either via:
        * Simple interpolation
            * may require guess of $\nu$ if orbital elements are being used
            * requires guess of $T$
        * Gravity Turn solution of reduced problem
            * may require guess of $\nu$ if orbital elements are being used
            * cannot be used to compute trajectories with stationary final states (surface to orbit only)
        * Warm start 
            * Requires $X$, $U$ and $T$ have solutions interally already
        * Externally provided trajectory
            * Must provide $X$ and $U$ of the right shape (maybe allow interpolation later)
            * Must provide $T$
    * $T_{min}$, $T_{max}$
    * multi stage?
        * Requires only that a list of vehicle definitions are provided instead of just one
        * Requires problem is surface to orbit only
        * Creates a special loop over what is essentially many single stage problems with a different minimization goal
        * May be harder to return results unless some standard is defined
        * May need special detection for the possiblility that the rocket is overbuilt and extra stages are not needed, this can be done using rudementary $\Delta v$ remaining calculation
        * New boundary conditions on control inputs are needed
* Solution Paramters:
    * N
    * solution tolerance
    * terminal state tolerance
    * maximum iterations
    * integration method
    * verbose
    * bound relax factor
    * nlp scaling method
    * mumps mem percent
* Outputs:
    * ???? All kinds of stuff
    * Oodles of noodles of info



To solve the following occurs:
* 2D Gravity Turn must be computed
    * Needs a target state vector which is compute from orbitial elements
        * Cartesian state to spherical
* All Gravity Turn states are rotated from spherical to spherical to the right position for an initial guess
    * Spherical states to cartesian states
    * Cartesian states to sphereical states
* If using orbital elements it is necessary to do the following:
    * h and e are computed from orbital elements
    * compute h and e symbolically from spherical position
        * symbolic spherical single state to cartesian
* For plotting purposes it is also desirable to have the following capability:
    * Compute orbital elements from cartesian state vectors
    * Sphereical state vectors to cartesian

Methods needed:

* Coordinate transforms
    * Basis change of state(s) function [DONE]
    * Symblic spherical state to cartesian state [DONE]
    * State rotation function [DONE]
    * Control rotation function [DONE]
* Orbital Element
    * From orbitial elements to pos, vel, h, e [DONE]
    * From pos vel to h, e, orbital elements [DONE]
    * Symbolic state to h, e [DONE]


In [1]:
import pyoptflight as pof
from pyoptflight import initialize as optinit
from pyoptflight import plotting as optplot
import numpy as np

In [2]:
kerbin = pof.Body("Kerbin")

vehicle = pof.Stage.load_vehicle('mintoc_single')

config = pof.SolverConfig(landing=False, 
                          T_min=0,
                          T_max = 700,
                          max_iter=250, 
                          solver_tol=1e-4, 
                          N=50, 
                          T_init=100,
                          q_max = 100,
                          integration_method='RK4')

x0 = pof.LatLngBoundary(lat=0,
                        lng=0,
                        alt=0,
                        v_eps=1e-6,
                        ub_lng=-180,
                        lb_lng=180,
                        body=kerbin,
                        config=config)
xf = pof.KeplerianBoundary(i=np.deg2rad(60),
                           Ω=np.deg2rad(15),
                           ω=0,
                           ha=80,
                           hp=80,
                        #    ν = np.deg2rad(100),
                           body=kerbin)

# single_stage_solver = pof.Solver(kerbin, [single_stage], config, x0, xf)
multi_stage_solver = pof.Solver(kerbin, vehicle, config, x0, xf)

In [3]:
multi_stage_solver.initialize_from_func(optinit.gravity_turn, {'skew': False})

The minimum cost is 0.2376889534464345 rad
This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     1633
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      986

Total number of variables............................:      299
                     variables with only lower bounds:      148
                variables with lower and upper bounds:      151
                     variables with only upper bounds:        0
Total number of equality constraints.................:      250
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  9.9870000e-01 1.46e+00 9.97e

In [4]:
multi_stage_solver.stats()

{'status': 'Solve_Succeeded',
 'success': True,
 'runtime': 1.3923606872558594,
 'iter_count': 22,
 'T': 556.1587085907552,
 'nsolves': 1}

In [5]:
# multi_stage_solver.config.integration_method = 'RK4'
# multi_stage_solver.config.solver_tol = 1e-6
# multi_stage_solver.extra_opts = {'ipopt.warm_start_init_point': 'yes'}
# multi_stage_solver.extra_opts = {'ipopt.hessian_approximation': 'limited-memory'}
# multi_stage_solver.warm_start = True
multi_stage_solver.extra_opts = {}

In [5]:
multi_stage_solver.solve()

RUNNING RK4 METHOD
End of constructor loop time: 0.18031740188598633
Construction of NLP: 0.78501296043396
This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     3512
Number of nonzeros in inequality constraint Jacobian.:       36
Number of nonzeros in Lagrangian Hessian.............:     2677

Total number of variables............................:      501
                     variables with only lower bounds:       49
                variables with lower and upper bounds:      305
                     variables with only upper bounds:        0
Total number of equality constraints.................:      350
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        6
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d|| 

In [ ]:
# RK4 base tol: 170, 74, 53, 384, 85, DNF (max iter) # No warm_start = False/no
# RK4 base tol: 170, 101ty, 139ty, 104 fn
# CVODES base tol: 189fn, 169tn, 119ty
# spral: 183, DNFty
# ma: Bugged!
# pardisomkl: MIA
# pardiso: Bugged! 

In [ ]:
import casadi as ca
J = ca.jacobian(ca.vertcat(*multi_stage_solver.G), multi_stage_solver.V)
sparsity_pattern = J.sparsity()

In [ ]:
print(sparsity_pattern)

In [ ]:
fig = optplot.plot_solutions(multi_stage_solver, 
                             show_orbit=True, 
                             size=(600, 600), 
                             ctrl_colorscale=True, 
                             ctrl_markers=True,
                             indices=[-1])
fig.show()

$$
\vec{D} = -\frac{1}{2}\rho C_D A ||\vec{v}||^2 \frac{\vec{v}}{||\vec{v}||}\\

\vec{D} = -\frac{1}{2}\rho C_D A ||\vec{v}|| \vec{v}\\

\vec{D} = -\frac{1}{2}\rho C_D A \sqrt{v_x^2+v_y^2+v_z^2} \vec{v}\\
$$


$$
\vec{L} = \frac{1}{2}\rho C_L A ||\vec{v}||^2 \frac{\vec{v}}{||\vec{v}||}\\
$$


$$
\hat{l} = \frac{\vec{v}\times(\vec{v}\times\hat{d})}{||\vec{v}\times(\vec{v}\times\hat{d}) + \epsilon||}
$$

$$
\text{proj}_{\vec{v}}\hat{d} = \frac{\hat{d}\cdot\vec{v}}{\vec{v}\cdot\vec{v}}\vec{v} = \frac{\hat{d}\cdot\vec{v}}{||\vec{v}||^2}\vec{v}
$$

$$
\hat{d} - \text{proj}_{\vec{v}}\hat{d} \\
\hat{d} - \frac{\hat{d}\cdot\vec{v}}{||\vec{v}||^2}\vec{v}
$$

$$
\vec{L} = \frac{1}{2}\rho C_L A ||\vec{v}||^2 \left(\hat{d} - \frac{\hat{d}\cdot\vec{v}}{||\vec{v}||^2}\vec{v}\right)
$$

$$
\hat{v}\times\frac{(\hat{v}\times\hat{d})}{\sin{\alpha}}
$$

In [ ]:
xp = x*np.cos(angle) + np.cross(axis, x)*np.sin(angle) + axis*np.dot(axis, x)*(1 - np.cos(angle))

In [ ]:
anlge = np.arctan2(np.dot(np.cross(a, b), axis), np.dot(a, b))

In [ ]:
np.arctan2(0, 0)

One idea is to use arccos to find the AoA then find the axis of rotation between d and v via cross product (can be zero vector) and then rotate d by pi/2 - AoA about axis using the above formula for vector rotation. 

In [ ]:
import casadi as ca
import numpy as np

vx = ca.SX.sym("vx")
vy = ca.SX.sym("vy")
vz = ca.SX.sym("vz")
v = ca.vertcat(vx, vy, vz)
dx = ca.SX.sym("dx")
dy = ca.SX.sym("dy")
dz = ca.SX.sym("dz")
d = ca.vertcat(dx, dy, dz)

n = ca.cross(v, ca.cross(d, v))
norm = ca.norm_2(n)

dtest = [0, -np.sin(0.0001), np.cos(0.0001)]
vtest = [0, 0, 1]

ftrue = ca.Function("ftrue",[vx, vy, vz, dx, dy, dz],[ca.vertcat(0, 0, 0)]) # True
ffalse = ca.Function("ffalse",[vx, vy, vz, dx, dy, dz],[n/norm]) # False
f_cond = ca.Function.if_else('f_cond', ftrue, ffalse)
func = ca.Function("func", [vx, vy, vz, dx, dy, dz], [f_cond(norm < 1e-15, vx, vy, vz, dx, dy, dz)])

In [ ]:
func(*vtest, *dtest)

In [ ]:
test_array = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
np.insert(test_array, [1, 2, 3], np.zeros((3)), axis=0)

In [ ]:
t1 = [1, 2, 3]
np.cumsum(t1)
sum(t1[0:0])

In [ ]:
np.append(test_array, np.array([np.zeros((3))]), axis=0)

In [ ]:
import numpy as np
test_arr = [[0, 1, 2, 3], [-1, 4, -2, 5]]
[1, *np.min(test_arr, axis=0).tolist()]
